Import packages

In [1]:
import pandas as pd

Set global variables

In [2]:
PATH_IN = 'data/output/'

SCENARIO = 'testing'
YEAR = '2024'

Read output data

In [3]:
if SCENARIO == 'testing':
    path_input = PATH_IN + SCENARIO + '/'
else:
    path_input = PATH_IN + SCENARIO + '/' + YEAR + '/'

output_df = pd.read_csv(path_input + 'output_time_series.csv', index_col=0)
output_df.index.name = 't'
print(output_df.columns)

Index(['power_price', 'gas_price', 'bhkw_bin', 'storage_bin', 'bhkw_gas',
       'bhkw_power', 'bhkw_heat', 'plant_power', 'storage_power',
       'storage_energy'],
      dtype='object')


Prepare class for Assets

In [4]:
class Asset:
    """Class for evaluating asset output data."""

    def __init__(self, asset_id) -> None:
        self.id: str = asset_id
        self.postprocess_data: dict = {}
        self.asset_data: pd.DataFrame = pd.DataFrame()

    def get_asset_data(self, output_df: pd.DataFrame):
        """ Gets all columns from output_df with the asset id in it. """

        for column_name in output_df.columns:
            if self.id in column_name:
                self.asset_data[column_name] = output_df[column_name]
        
        if self.asset_data.empty:
            print(f'Found no data for {self.id}.')
    
    def calc_costs(self, output_df: pd.DataFrame):
        """ Calculates costs for the asset."""

        for column in self.asset_data.columns:
            if '_gas' in column:
                self.asset_data[f'{self.id}_gas_cost'] = (
                    self.asset_data[f'{self.id}_gas'] 
                    * output_df['gas_price']
                )

            elif '_power' in column:
                self.asset_data[f'{self.id}_power_cost'] = (
                    self.asset_data[f'{self.id}_power'].where(
                    self.asset_data[f'{self.id}_power'] <= 0                    
                    )
                    * output_df['power_price']
                )

                self.asset_data[f'{self.id}_power_revenue'] = (
                    self.asset_data[f'{self.id}_power'].where(
                    self.asset_data[f'{self.id}_power'] >= 0                    
                    )
                    * output_df['power_price']
                ) 

    def calc_postprocess_data(self):
        """ Calculates the sum of all columns in the asset data."""

        for column in self.asset_data.columns:
            column_sum = self.asset_data[column].sum()
            key_name = column.replace((self.id + '_'), '')
            self.postprocess_data[key_name] = column_sum

Calculate overview data for assets

In [5]:
assets_dict = {}
for asset in ['bhkw', 'storage', 'plant', 'price']:
    assets_dict[asset] = Asset(asset)
    assets_dict[asset].get_asset_data(output_df)
    assets_dict[asset].calc_costs(output_df)
    assets_dict[asset].calc_postprocess_data()


postprocess_dict = {}
for asset in assets_dict.keys():
    postprocess_dict[asset] = assets_dict[asset].postprocess_data

print(postprocess_dict)

df = pd.DataFrame(postprocess_dict)
print(df)

{'bhkw': {'bin': 10.0, 'gas': 38.8, 'power': 16.0, 'heat': 16.0, 'gas_cost': 679.1999999999999, 'power_cost': 0.0, 'power_revenue': 769.0}, 'storage': {'bin': 0.0, 'power': 0.0, 'energy': 0.0, 'power_cost': 0.0, 'power_revenue': 0.0}, 'plant': {'power': 16.0, 'power_cost': 0.0, 'power_revenue': 769.0}, 'price': {'power_price': 692.0, 'gas_price': 420.0}}
                bhkw  storage  plant  price
bin             10.0      0.0    NaN    NaN
gas             38.8      NaN    NaN    NaN
power           16.0      0.0   16.0    NaN
heat            16.0      NaN    NaN    NaN
gas_cost       679.2      NaN    NaN    NaN
power_cost       0.0      0.0    0.0    NaN
power_revenue  769.0      0.0  769.0    NaN
energy           NaN      0.0    NaN    NaN
power_price      NaN      NaN    NaN  692.0
gas_price        NaN      NaN    NaN  420.0


Define function for checking the objectiv value

In [6]:
def check_objective_value(output_dict):
    result = (output_dict['bhkw']['gas_cost'] 
              - output_dict['plant']['power_revenue']
              )
    obj_value = (pd.read_csv(path_input + 'results.csv', index_col=0)).iloc[0].values[0]
    
    if result == obj_value:
        print('Objective value and calculated value are the same.')
        print(f'{obj_value} == {result}')
    
    else:
        print('Objective value and calculated value are not identical!')
        print(f'{obj_value} =/= {result}')

In [7]:
check_objective_value(postprocess_dict)

Objective value and calculated value are the same.
-89.80000000000007 == -89.80000000000007
